In [18]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf,countDistinct,sproc,udf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
import pandas as pd
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
import sys
import json
import platform
import os,requests
from pathlib import Path
import glob
from src.DataValidationContext import *


from snowflake.snowpark import version
print(version.VERSION)
session = Session.builder.configs(snowflake_conn_prop).create()

print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(1, 11, 1)
[Row(CURRENT_WAREHOUSE()='COMPUTE', CURRENT_DATABASE()='CITIBIKE_2', CURRENT_SCHEMA()='PUBLIC')]


In [19]:

PROJECT_HOME_DIR = '.'
LOCAL_TEMP_DIR = os.path.join(PROJECT_HOME_DIR, 'temp') 
LOCAL_LIB_DIR = os.path.join(LOCAL_TEMP_DIR, 'libs')
LOCAL_TARFile_DIR = os.path.join(LOCAL_TEMP_DIR, 'tarfiles')
LIB_URLS = [
    #'https://files.pythonhosted.org/packages/9f/57/1539d783553f3d67cea1b55d7fe494373c5c0c9af689d4c0e0c2d3197739/great_expectations-0.15.17-py3-none-any.whl'
    'https://files.pythonhosted.org/packages/8e/9d/cecb12289f7967b15facf550a0bbb9c1e910968c3a61b91fd8cdb80aeb3c/great_expectations-0.15.14.tar.gz'
    
]

for lib_url in LIB_URLS:
    # get the file name, from the url
    splits = lib_url.split('/')
    tot_splits = len(splits)
    target_file = splits[-1]
    
    local_lib_fl = f'{LOCAL_TARFile_DIR}/{target_file}'
    print(local_lib_fl)

    # Create a local directory for TAR and extracting tar..
    Path(LOCAL_TARFile_DIR).mkdir(parents=True, exist_ok=True)
    print(f'Create local dir: {LOCAL_TARFile_DIR}')

    Path(LOCAL_LIB_DIR).mkdir(parents=True, exist_ok=True)
    print(f'Create local dir: {LOCAL_LIB_DIR}')

    print(f'Downloading library from PyPI to {LOCAL_TARFile_DIR} ...')
    with open(local_lib_fl, "wb") as f:
        r = requests.get(lib_url)
        f.write(r.content)

        
# Extract GE tar file

import tarfile
file = tarfile.open(local_lib_fl)
print(f'Started Extracting GE tar file to {LOCAL_TARFile_DIR} ...')
file.extractall(f'{LOCAL_LIB_DIR}/ge')
file.close()
print(f'Done extracting GE tar file to {LOCAL_TARFile_DIR} ...')
                                    

./temp/tarfiles/great_expectations-0.15.14.tar.gz
Create local dir: ./temp/tarfiles
Create local dir: ./temp/libs
Started Extracting GE tar file to ./temp/tarfiles ...
Done extracting GE tar file to ./temp/tarfiles ...


In [20]:
# Getting the path for the great_expectation folder after the tar file is extracted.

import glob
ge_import_path=''
for result in glob.iglob('./temp/libs/ge/great_expectations*'):
    ge_import_path=result+'/great_expectations'
print(ge_import_path)


./temp/libs/ge/great_expectations-0.15.14/great_expectations


In [35]:
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
from src.DataValidationContext import GEDataValidationContext
from src.BatchRequest import getBatchRequest
from src.Expectations import  createExpectationSuite, createExpectations
from src.RunLoadExpectations import runExpectaionValidation,loadValidationToDB
from dotenv import load_dotenv
load_dotenv()

import json
import os


session.sql("create or replace stage phani_greatexpectation").collect()
session.clear_packages()
session.add_packages('packaging','python-dotenv','pyparsing','pandas','pycryptodomex','boto3','tzlocal','tqdm','requests','ruamel.yaml','ipython','jsonpatch','mistune','jinja2','jsonschema','scipy','altair','Click','colorama','cryptography','snowflake-snowpark-python','sqlalchemy','chardet','asn1crypto')
session.clear_imports()
session.add_import(ge_import_path)
session.add_import('src')
session.add_import('configs')

@sproc(session=session,name="GEValidationResults", replace=True, 
       return_type=StringType(),
       input_types=[StringType(),StringType(),StringType()],
       is_permanent=True, stage_location='@phani_greatexpectation/ge_AllLibs')

def generateGEValidationResults(session: Session, db_name: str, schema_name: str ,table_name: str) -> str:
        
    from pathlib import Path
    import os ,sys ,json ,tarfile, dotenv
    
    expecationsuitename = "expecationsuitename"
    checkpointname = "checkpointname"
    
    sftablename = f'{db_name}.VALIDATION.{table_name}_VALIDATION_RAW'

    #Creating GE context inside code
    ge=GEDataValidationContext(table_name)
    context=ge.getContext()
    
    # Creating the Pandas DataFrame from Snowpark DF
    query = f' SELECT * FROM {db_name}.{schema_name}.{table_name} LIMIT 10000'
    pd_df = session.sql(query).to_pandas()   

    # Getting the batch request used while creating and running validation on expectations
    local_batch_request=getBatchRequest(context,table_name,pd_df)
    
    #Creating GE expectation Suite
    createExpectationSuite(context,expecationsuitename)

    #Creating GE expecations
    createExpectations(session, context,expecationsuitename,local_batch_request,pd_df, table_name)
    
    #Running GE validation 
    res=runExpectaionValidation(context,checkpointname,local_batch_request,expecationsuitename,table_name)
    
    #Loading validation result to Snowflake table. Using append option while writing the data to the table
    loadValidationToDB(session,res,sftablename)
      
    return f'VALIDATION RESULTS SUCCESSFULLY STORED ON {sftablename}'.upper()

    

The version of package 'packaging' in the local environment is 23.2, which does not fit the criteria for the requirement 'packaging'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'python-dotenv' in the local environment is 1.0.0, which does not fit the criteria for the requirement 'python-dotenv'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'pyparsing' in the local environment is 3.1.1, which does not fit the criteria for the requirement 'pyparsing'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'pandas' in the local environment is 2.1.4, which does not fit the criteria for the requirement 'pandas'. Your UDF might not work when the package version is different between the server and your local environment.
Package 'pycryptodomex'

In [36]:
#Calling SP
session.sql("call GEValidationResults('CITIBIKE_2','PUBLIC','TRIPS')").collect()

[Row(GEVALIDATIONRESULTS='VALIDATION RESULTS SUCCESSFULLY STORED ON CITIBIKE_2.VALIDATION.TRIPS_VALIDATION_RAW')]